# [실습] OpenAI API를 이용한 검색 에이전트 만들기

OpenAI Chat API를 이용해 검색 결과를 요약하거나, 질의 응답을 수행하는 프로그램을 만들어 보겠습니다.


## 기본 라이브러리 설치 및 환경 변수 설정   

이번 실습부터는, 파일에 직접 API 키를 입력하는 대신   
별도의 환경 변수 파일을 작성하여 불러오는 `dotenv`를 사용합니다.

In [1]:
!pip install openai tiktoken dotenv tavily-python -q
# -q: 조용히 실행


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### .env 파일 만들기
현재 폴더에서, '.env' 파일을 만들고 파일 내용에   
`OPENAI_API_KEY="API 키"`를 입력하세요.

load_dotenv() 함수는 `.env' 파일에서 환경 변수 설정을 가져옵니다.   


파일명을 직접 입력하거나,    
override=True를 통해 기존 설정을 덮어쓸 수 있습니다.

In [2]:
from dotenv import load_dotenv

#.env 파일에서 환경 변수 설정 로드
load_dotenv(override=True)

# load_dotenv('파일이름', override=True)

True

정상적으로 키가 저장되고 불러와진 경우, API 접속이 가능합니다.

In [4]:
import openai
import os

client = openai.OpenAI()

# API 키 검증하기
try: client.models.list(); print("OPENAI_API_KEY가 정상적으로 설정되어 있습니다.")
except:  print(f"API 키가 유효하지 않습니다!")

OPENAI_API_KEY가 정상적으로 설정되어 있습니다.


기존의 Chat Completion API는 아래와 같은 함수로 간략하게 나타낼 수 있습니다.

In [5]:
# Test
def chat(msg, model = 'gpt-4.1-mini'):
    messages = [{'role':'user','content':msg}]

    response = client.chat.completions.create(
        model = model,
        messages = messages,
        temperature = 0.2,
        max_tokens = 4096
    )
    return response.choices[0].message.content

chat("안녕?")

'안녕하세요! 어떻게 도와드릴까요?'

## 검색 API 준비하기

다음의 검색 API를 사용하겠습니다.  

1. 네이버의 뉴스 검색 API (일일 1,500회 무료)

   네이버 로그인 후, 아래의 링크에서 API 키를 발급합니다.   
   (https://developers.naver.com/apps/#/register)
   - 사용 API: 검색
   - 비로그인 오픈 API 서비스 환경: file://test
   - 어플리케이션 이름: test


2. Tavily Search (월간 1,000회 무료)   
   구글 계정으로 아래 사이트에 로그인하여 API 키를 발급합니다.   
   - (https://app.tavily.com/)


## 1. 네이버 검색 API 호출 함수 구성

API 문서나 호출 결과를 LLM에 입력하면, 간단하게 만들 수 있습니다.   
(네이버 API는 충분히 모델에 학습되어 있어 API 문서가 없어도 됩니다)

In [7]:
prompt = '''
네이버 뉴스 API를 이용해 크롤링을 수행하는 파이썬 코드를 작성해 주세요.
검색어가 `query` 인수로 주어지면, 관련도순으로 검색된 뉴스 기사 30개의 정보를
아래 포맷의 문자열로 변환하는 함수 get_news(query) 를 만들어 주세요.

.env에서 환경 변수를 load_dotenv(override=True) 옵션으로 불러옵니다.
---
제목: (뉴스 제목)
URL: (뉴스 링크)
내용: (뉴스 내용)
---

# 검색어
query = '생성형 AI'


# 검색 결과를 return하는 함수
def get_news(query):
...

result = get_news(query)
print(result)
'''



In [8]:
response = client.chat.completions.create(
    model = "gpt-4.1-mini",
    messages = [{'role':'system', 'content':'설명 없이 코드만 출력하세요.'},
          {'role':'user', 'content': prompt}],

    temperature =  0,
    max_tokens = 2000,

    seed = 2025

)
print(response.choices[0].message.content)

```python
import os
import requests
from dotenv import load_dotenv

load_dotenv(override=True)

def get_news(query):
    client_id = os.getenv('NAVER_CLIENT_ID')
    client_secret = os.getenv('NAVER_CLIENT_SECRET')
    if not client_id or not client_secret:
        return "NAVER_CLIENT_ID or NAVER_CLIENT_SECRET not set in environment variables."

    url = "https://openapi.naver.com/v1/search/news.json"
    headers = {
        "X-Naver-Client-Id": client_id,
        "X-Naver-Client-Secret": client_secret
    }
    params = {
        "query": query,
        "display": 30,
        "start": 1,
        "sort": "sim"
    }

    response = requests.get(url, headers=headers, params=params)
    if response.status_code != 200:
        return f"Error: {response.status_code}"

    items = response.json().get('items', [])
    result = []
    for item in items:
        title = item.get('title', '').replace('<b>', '').replace('</b>', '')
        link = item.get('originallink') or item.get('link', ''

env 파일의 내용을 수정하고, 아래에 코드를 복사하여 실행하고 결과를 확인합니다.

In [9]:
import os
import requests
from dotenv import load_dotenv

load_dotenv(override=True)

def get_news(query):
    client_id = os.getenv('NAVER_CLIENT_ID')
    client_secret = os.getenv('NAVER_CLIENT_SECRET')
    if not client_id or not client_secret:
        raise ValueError("NAVER_CLIENT_ID and NAVER_CLIENT_SECRET must be set in .env")

    url = "https://openapi.naver.com/v1/search/news.json"
    headers = {
        "X-Naver-Client-Id": client_id,
        "X-Naver-Client-Secret": client_secret
    }
    params = {
        "query": query,
        "display": 30,
        "start": 1,
        "sort": "sim"
    }

    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    data = response.json()

    result = []
    for item in data.get('items', []):
        title = item.get('title', '').replace('&quot;', '"').replace('&lt;', '<').replace('&gt;', '>').replace('<b>', '').replace('</b>', '')
        link = item.get('link', '')
        description = item.get('description', '').replace('&quot;', '"').replace('&lt;', '<').replace('&gt;', '>').replace('<b>', '').replace('</b>', '')
        result.append(f"---\n제목: {title}\nURL: {link}\n내용: {description}\n---")

    return '\n'.join(result)


query = '생성형 AI'
result = get_news(query)
print(result)

---
제목: 네이버·업스테이지·SKT·NC AI·LG AI연, '국대 AI' 만든다
URL: https://n.news.naver.com/mnews/article/001/0015547564?sid=105
내용: 생성형 AI의 모태가 된 트랜스포머 기술을 고도화한 '포스트-트랜스포머 AI 모델'로 K-AI 서비스를 구현한다는 목표로, 누구나 활용할 수 있는 범용 AI 에이전트 등 기업 대 고객(B2C) 서비스, 제조·자동차·게임·로봇 등... 
---
---
제목: LG, 국가대표 AI 정예팀 합류…'K-엑사원' 프로젝트 가동
URL: https://n.news.naver.com/mnews/article/031/0000954287?sid=101
내용: 글로벌 AI 선도국으로 도약할 수 있도록 최선을 다할 것”이라고 밝혔다. 한편, LG AI연구원은 국내 대학원생들에게 생성형 AI 연구 경험을 쌓을 기회를 제공하기 위해 공모형 인턴제도를 확대한다고 밝혔다.
---
---
제목: LG 컨소시엄, '국가대표 AI' 개발 맡아…'K-엑사원' 프로젝트 가동
URL: https://n.news.naver.com/mnews/article/417/0001093294?sid=101
내용: 한편 LG AI연구원은 국내 인재 양성에도 적극 나선다. 기존 운영 중인 '공모형 인턴제도'를 확대해 더 많은 국내 대학원생에게 글로벌 수준의 생성형 AI 연구 경험을 제공할 예정이다.
---
---
제목: '국대 AI' 만든다…네이버·업스테이지·SKT·NC·LG 선정
URL: https://n.news.naver.com/mnews/article/055/0001281119?sid=102
내용: 생성형 AI의 모태가 된 트랜스포머 기술을 고도화한 '포스트-트랜스포머 AI 모델'로 K-AI 서비스를 구현한다는 목표로, 누구나 활용할 수 있는 범용 AI 에이전트 등 기업 대 고객(B2C) 서비스, 제조·자동차·게임·로봇 등... 
---
---
제목: [속보]네이버·업스테이지·SKT·NC AI·L

## 2. Tavily API 호출 함수 구성   

Tavily는 AI 최적화 검색 엔진입니다.   
다양한 활용이 가능합니다.   

https://app.tavily.com/playground

.env 파일에 TAVILY_API_KEY를 추가합니다.

In [10]:
from tavily import TavilyClient

load_dotenv(override=True)

tavily = TavilyClient()
response = tavily.search(
    query = "Qwen 3",
    max_results = 3,
)
response

{'query': 'Qwen 3',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://qwenlm.github.io/blog/qwen3/',
   'title': 'Qwen3: Think Deeper, Act Faster | Qwen',
   'content': 'Our flagship model, **Qwen3-235B-A22B**, achieves competitive results in benchmark evaluations of coding, math, general capabilities, etc., when compared to other top-tier models such as DeepSeek-R1, o1, o3-mini, Grok-3, and Gemini-2.5-Pro. Additionally, the small MoE model, **Qwen3-30B-A3B**, outcompetes QwQ-32B with 10 times of activated parameters, and even a tiny model like Qwen3-4B can rival the performance of Qwen2.5-72B-Instruct. Qwen3 models are supporting **119 languages and dialects**. We have optimized the Qwen3 models for coding and agentic capabilities, and also we have strengthened the support of MCP as well. This data was generated by the enhanced thinking model from the second stage, ensuring a seamless blend of reasoning and quick response capabilities. model_n

네이버 검색과 동일하게, 함수로 변환해 보겠습니다.

In [11]:
response['results']

[{'url': 'https://qwenlm.github.io/blog/qwen3/',
  'title': 'Qwen3: Think Deeper, Act Faster | Qwen',
  'content': 'Our flagship model, **Qwen3-235B-A22B**, achieves competitive results in benchmark evaluations of coding, math, general capabilities, etc., when compared to other top-tier models such as DeepSeek-R1, o1, o3-mini, Grok-3, and Gemini-2.5-Pro. Additionally, the small MoE model, **Qwen3-30B-A3B**, outcompetes QwQ-32B with 10 times of activated parameters, and even a tiny model like Qwen3-4B can rival the performance of Qwen2.5-72B-Instruct. Qwen3 models are supporting **119 languages and dialects**. We have optimized the Qwen3 models for coding and agentic capabilities, and also we have strengthened the support of MCP as well. This data was generated by the enhanced thinking model from the second stage, ensuring a seamless blend of reasoning and quick response capabilities. model_name = "Qwen/Qwen3-30B-A3B" python -m sglang.launch_server --model-path Qwen/Qwen3-30B-A3B --reas

In [12]:
def tavily_search(query, max_results = 5):
    response = tavily.search(
    query = query,
    max_results = max_results,
    )
    # results 값을 하나의 문자열로 결합
    return ' \n---\n '.join(["Title: {title} \n URL: {url} \n Content: {content}".format(**i) for i in response['results']])

result = tavily_search("Qwen 3")
print(result)

Title: Qwen3: Think Deeper, Act Faster | Qwen 
 URL: https://qwenlm.github.io/blog/qwen3/ 
 Content: Our flagship model, **Qwen3-235B-A22B**, achieves competitive results in benchmark evaluations of coding, math, general capabilities, etc., when compared to other top-tier models such as DeepSeek-R1, o1, o3-mini, Grok-3, and Gemini-2.5-Pro. Additionally, the small MoE model, **Qwen3-30B-A3B**, outcompetes QwQ-32B with 10 times of activated parameters, and even a tiny model like Qwen3-4B can rival the performance of Qwen2.5-72B-Instruct. Qwen3 models are supporting **119 languages and dialects**. We have optimized the Qwen3 models for coding and agentic capabilities, and also we have strengthened the support of MCP as well. This data was generated by the enhanced thinking model from the second stage, ensuring a seamless blend of reasoning and quick response capabilities. model_name = "Qwen/Qwen3-30B-A3B" python -m sglang.launch_server --model-path Qwen/Qwen3-30B-A3B --reasoning-parser qw

<br><br><br>
## Tool Calling : 외부 도구 사용하기   

LLM은 Tool Calling을 통해 검색 기능을 관리할 수 있겠습니다.  



두 개의 함수를 LLM이 이해할 수 있는 JSON 형태로 변환합니다.   
이를 스키마(Schema)라고 하는데, Pydantic을 이용해 쉽게 변환할 수 있습니다.

-------


In [13]:
from openai import pydantic_function_tool
from pydantic import BaseModel, Field

class Naver_News_Search(BaseModel):
    """query를 이용하여 네이버 뉴스를 관련도 순으로 검색합니다."""
    query: str= Field(description="""검색 키워드
규칙:
1. 최대 2개 단어로 구성
2. 불필요한 조사나 형용사 제외
3. 핵심 명사만 포함

예시:
- (좋음) "개봉영화", "영화"
- (나쁨) "새로 개봉한 영화", "요즘 인기있는 영화"
""")

class Tavily_Search(BaseModel):
    """Tavily 검색 엔진을 이용해 일반 텍스트 웹 검색을 수행합니다."""
    query: str = Field(description="""검색 키워드(영어로 변환 선호, 질문으로 전달할 것)""")
    max_results: int = Field(description="""검색 결과 문서의 수,
특별한 요청이 없으면 5, 최소 3에서 최대 20개""")

tools = [pydantic_function_tool(Naver_News_Search), pydantic_function_tool(Tavily_Search)]
tools

[{'type': 'function',
  'function': {'name': 'Naver_News_Search',
   'strict': True,
   'parameters': {'description': 'query를 이용하여 네이버 뉴스를 관련도 순으로 검색합니다.',
    'properties': {'query': {'description': '검색 키워드\n규칙:\n1. 최대 2개 단어로 구성\n2. 불필요한 조사나 형용사 제외\n3. 핵심 명사만 포함\n\n예시:\n- (좋음) "개봉영화", "영화"\n- (나쁨) "새로 개봉한 영화", "요즘 인기있는 영화"\n',
      'title': 'Query',
      'type': 'string'}},
    'required': ['query'],
    'title': 'Naver_News_Search',
    'type': 'object',
    'additionalProperties': False},
   'description': 'query를 이용하여 네이버 뉴스를 관련도 순으로 검색합니다.'}},
 {'type': 'function',
  'function': {'name': 'Tavily_Search',
   'strict': True,
   'parameters': {'description': 'Tavily 검색 엔진을 이용해 일반 텍스트 웹 검색을 수행합니다.',
    'properties': {'query': {'description': '검색 키워드(영어로 변환 선호, 질문으로 전달할 것)',
      'title': 'Query',
      'type': 'string'},
     'max_results': {'description': '검색 결과 문서의 수,\n특별한 요청이 없으면 5, 최소 3에서 최대 20개',
      'title': 'Max Results',
      'type': 'integer'}},
    'required': ['query

`tools`에는 Docstring과 입력 형식 등의 정보가 포함되며, 이는 시스템 프롬프트에 들어갑니다.

Tool 정보는 기존의 Chat Completions API에 추가합니다.   

In [14]:
def search_bot(messages, stream=False, model = 'gpt-4.1-mini'):

    response = client.chat.completions.create(
        model = model,
        messages = messages,

        #### 사용할 툴 목록 전달
        tools = tools,
        tool_choice = 'auto',
        # 'auto' : 자율적 툴 판단
        # 'none'이면 툴 사용하지 않음
        # 'required'면 무조건 툴 사용
        ####

        temperature= 0.1,
        max_tokens= 1024,

        stream = stream
        # 기본값은 False--> 일반 출력
        # stream==True면 스트리밍 출력
    )

    # Streaming 여부에 따라 출력 다르게 하기
    if stream: return response

    return response.choices[0].message

LLM은 함수의 구조와 설명을 이용하여 어떤 함수를 써야 하는지를 판단하고 출력합니다.

In [16]:
from rich import print as rprint

# tool 사용 필요 없음
result = search_bot([
    {
        "role": "user",
        "content": """안녕하세요! 오늘 날씨가 좋네요."""
    }
])
rprint(result)

ChatCompletionMessage(
    content='안녕하세요! 오늘 날씨가 좋다니 기분이 좋아지네요. 오늘 특별히 계획하신 일이 있으신가요?',
    refusal=None,
    role='assistant',
    annotations=[],
    audio=None,
    function_call=None,
    tool_calls=None
)

In [23]:
# tool 사용 필요함
tool_call_result = search_bot([
    {
        "role": "user",
        "content": """
넷플릭스 신작 추천해줘. 무슨 툴 어떻게 쓸지를 먼저 설명하고 작업해.
"""
    }
])
rprint(tool_call_result)

ChatCompletionMessage(
    content='넷플릭스 신작 추천을 위해서는 최신 넷플릭스 신작 정보를 얻는 것이 중요합니다. 이를 위해 다음과 같은 
접근 방식을 사용할 수 있습니다.\n\n1. 네이버 뉴스 검색 (functions.Naver_News_Search):\n   - "넷플릭스 신작" 
키워드로 최신 뉴스 기사를 검색하여 최근에 나온 넷플릭스 신작 정보를 파악할 수 있습니다.\n   - 뉴스 기사에는 신작 
소개, 평점, 인기 정도 등이 포함될 수 있어 추천에 유용합니다.\n\n2. 일반 웹 검색 (functions.Tavily_Search):\n   - 
영어로 "latest Netflix releases" 또는 "new Netflix shows 2024" 같은 키워드로 최신 넷플릭스 신작 정보를 검색할 수 
있습니다.\n   - 다양한 리뷰 사이트, 블로그, 공식 발표 등을 통해 신작 목록과 추천작을 얻을 수 있습니다.\n\n이 두 
가지 방법을 병행하여 최신 넷플릭스 신작 정보를 수집한 후, 그 중에서 추천할 만한 작품을 선정해 드리겠습니다.\n\n이제
두 가지 검색을 병행 실행하여 최신 넷플릭스 신작 정보를 수집하겠습니다.',
    refusal=None,
    role='assistant',
    annotations=[],
    audio=None,
    function_call=None,
    tool_calls=[
        ChatCompletionMessageToolCall(
            id='call_JjaoU0vNoXVzck6pFbXFwqW6',
            function=Function(arguments='{"query": "넷플릭스 신작"}', name='Naver_News_Search'),
            type='function'
        ),
        ChatCompletionMessageToolCall(
            id='call_L2oBjDLcD2XPxLsg6OUC72Pt',
            function=Function(
                arguments='{"query": "latest Netflix releases", "max_results": 5}',
                name='Tavily_Search'
            ),
            type='function'
        )
    ]
)

In [24]:
# tool 사용 필요함
tool_call_result = search_bot([
    {
        "role": "user",
        "content": """
요즘 개봉한 영화 뭐 있어? 네이버 검색으로 찾아줘.
"""
    }
])
rprint(tool_call_result)

ChatCompletionMessage(
    content=None,
    refusal=None,
    role='assistant',
    annotations=[],
    audio=None,
    function_call=None,
    tool_calls=[
        ChatCompletionMessageToolCall(
            id='call_xNtyBaDv0nXymHg1l6zzO8nQ',
            function=Function(arguments='{"query":"개봉영화"}', name='Naver_News_Search'),
            type='function'
        )
    ]
)

`tool_calls`의 내용은 LLM의 함수 요청으로 볼 수 있습니다.

LLM이 입력을 판단하여, 실행할 함수의 정보를 `tool_calls`에 돌려줍니다.   
(Tool 실행은 여러 개일 수 있습니다.)

In [26]:
rprint(tool_call_result.tool_calls)

[
    ChatCompletionMessageToolCall(
        id='call_xNtyBaDv0nXymHg1l6zzO8nQ',
        function=Function(arguments='{"query":"개봉영화"}', name='Naver_News_Search'),
        type='function'
    )
]

tool_calls의 구성 요소는 다음과 같습니다.
- id: tool call의 id로, 해당 id에 실행 결과를 연결할 수 있습니다.
- function : arguments와 name을 통해 실행할 툴의 이름과 매개변수를 전달합니다.

In [27]:
(tool_call_result.tool_calls[0].id,
tool_call_result.tool_calls[0].function.name,
tool_call_result.tool_calls[0].function.arguments)

('call_xNtyBaDv0nXymHg1l6zzO8nQ', 'Naver_News_Search', '{"query":"개봉영화"}')

LLM에게 툴 실행 결과를 전달합니다.   
OpenAI API에서는 **User-AI(Tool 요청)-Tool** 을 아래와 같이 전달합니다.

In [28]:
result = search_bot([
    {
        "role": "user",
        "content": """
요즘 새로 개봉한 영화는 무엇이 있나요? 네이버 검색으로 알려주세요.
"""
    },
    tool_call_result, # tool_calls 메시지 (ChatCompletionMessage)
    # AI Message (tool call이 있는)
    {
        "role": "tool",
        "content": """
미션 임파서블: 파이널 레코닝, 플로리다 프로젝트(재개봉), 걸어도 걸어도(재개봉), 씨너스(죄인들)""",
        "tool_call_id":tool_call_result.tool_calls[0].id
        # ID: tool_call_result와 연결하는 역할
    }
])

rprint(result)

ChatCompletionMessage(
    content='요즘 새로 개봉한 영화로는 "미션 임파서블: 파이널 레코닝"이 있습니다. 또한 재개봉 영화로는 "플로리다 
프로젝트"와 "걸어도 걸어도"가 있으며, "씨너스(죄인들)"도 개봉했습니다. 더 궁금한 점 있으시면 알려주세요!',
    refusal=None,
    role='assistant',
    annotations=[],
    audio=None,
    function_call=None,
    tool_calls=None
)

이번에는 실제 함수를 실행하여 전달해 보겠습니다.

In [29]:
name = tool_call_result.tool_calls[0].function.name
argument = tool_call_result.tool_calls[0].function.arguments

name,argument

('Naver_News_Search', '{"query":"개봉영화"}')

툴의 이름과 매개변수는 문자열로 주어집니다.  
이를 파싱하고 변환하여 실제 실행 결과와 연결합니다.


In [30]:
# 문자열을 Dict로 바꾸기
import json
example = '{"query":"영화"}'
example_dict = json.loads(example)

print(example_dict)
print(type(example_dict))

{'query': '영화'}
<class 'dict'>


In [31]:
# Tool 이름과 실제 함수 Mapping
available_functions = {'Naver_News_Search': get_news, 'Tavily_Search': tavily_search}

# name = 'Naver_News_Search'
available_functions[name]

<function __main__.get_news(query)>

In [32]:
# Dictionary를 함수의 매개변수로 전달하기

example_dict = {'query': '영화'}

get_news(**example_dict)

'---\n제목: 영화 \'극한직업\' 배우 송영규 차 안에서 숨진 채 발견\nURL: https://n.news.naver.com/mnews/article/001/0015547151?sid=102\n내용: 영화 \'극한직업\'에서 최반장 역으로 주목받았던 배우 송영규씨가 숨진 채 발견돼 경찰이 수사 중이다. 4일... 1994년 어린이 뮤지컬 \'머털도사\'로 데뷔한 송씨는 최근까지 각종 영화와 드라마, 연극 등에 출연했다. 2019년... \n---\n---\n제목: 영화 \'극한직업\' 배우 송영규, 차에서 숨진 채 발견\nURL: https://n.news.naver.com/mnews/article/079/0004052241?sid=102\n내용: 영화 \'극한직업\'에 출연하는 등 다양한 활동을 벌였던 배우 송영규씨가 숨진 채 발견됐다. 4일 경찰에... 송씨는 1994년 어린이 뮤지컬로 데뷔해 최근까지 각종 영화와 드라마, 연극 등에 출연했다. 2019년 영화... \n---\n---\n제목: 영화 \'극한직업\' 배우 송영규 숨진 채 발견\nURL: https://n.news.naver.com/mnews/article/277/0005632230?sid=102\n내용: 영화 \'극한직업\'에서 최반장 역으로 주목받았던 배우 송영규씨가 숨진 채 발견돼 경찰이 수사 중이다. 4일... 1994년 어린이 뮤지컬 \'머털도사\'로 데뷔한 송씨는 최근까지 각종 영화와 드라마, 연극 등에 출연했다. 2019년... \n---\n---\n제목: 영화 \'극한직업\' 최 반장역 송영규 숨진 채 발견\nURL: https://n.news.naver.com/mnews/article/469/0000879703?sid=102\n내용: 영화 \'극한직업\'의 최 반장으로 잘 알려진 배우 송영규씨가 숨진 채 발견돼 경찰이 수사 중이다. 4일 경기... 1994년 어린이 뮤지컬 \'머털도사\'로 데뷔한 송씨는 각종 영화와 드라마, 연극 등에서 얼굴을 알렸다. 1,000만...

In [34]:
# 3개 코드 모두 사용하면?

print(available_functions[name](**json.loads(argument)))

---
제목: ‘좀비딸’ 개봉 5일 만에 180만 돌파…올해 최고 흥행 속도
URL: https://n.news.naver.com/mnews/article/028/0002759452?sid=103
내용: 예매율을 보면, ‘극장판 귀멸의 칼날: 무한성편’이 예매량 9만999명(20.5%)으로 ‘좀비딸’(19.1%)을 앞섰다. 지난달 18일 일본에서 개봉한 이 영화는 최단기간 100억엔 수입을 돌파했다. 한국에서는 오는 22일 개봉한다.
---
---
제목: 영화 '극한직업' 드라마 '카지노' 배우 송영규 숨진 채 발견
URL: https://n.news.naver.com/mnews/article/025/0003459741?sid=102
내용: 2019년 개봉한 영화 ‘극한직업’에서 최반장 역을 맡는 등 20여편이 넘는 영화에 출연했다. 온라인동영상 서비스(OTT) 드라마 ‘수리남(2022)’과 ‘카지노(2022)’ 등 수십 편의 드라마에서도 인상 깊은 연기를 펼쳤다.... 
---
---
제목: 난징대학살부터 731부대까지…中 '일제 항전' 영화 줄줄이 개봉
URL: https://n.news.naver.com/mnews/article/001/0015547149?sid=104
내용: 다룬 영화가 줄줄이 개봉하고 있다. 4일 관영매체 신화통신 등에 따르면 일제의 중국 침략 당시 세균전·생체실험을 했던 731부대를 다룬 영화 '731'이 중일 간 만주사변이 발발했던 9월 18일에 맞춰 정식 상영에... 
---
---
제목: 중국, 2차 세계대전 승전 80주년 맞아…‘반일 영화’ 줄개봉
URL: https://n.news.naver.com/mnews/article/021/0002727115?sid=104
내용: 중국산둥망 캡처 연합뉴스 중국에서 올해 제2차 세계대전 승리 80주년을 맞아 난징대학살이나 731부대 등 일제의 침략을 다룬 ‘반일 영화’가 줄줄이 개봉하고 있다. 박스오피스 1위에 오른 영화 ‘난징사진관’은... 
---
---
제목: 中 영화 ‘731’ 

이제 Tool 요청에서 검색을 연결해 보겠습니다.

In [35]:
# tool 사용 필요함
tool_call_result = search_bot([
    {
        "role": "user",
        "content": """
요즘 새로 개봉한 영화는 무엇이 있나요? 네이버 검색으로 알려주세요.
"""
    }
])
tool_call_result

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_kStcA0GjcrAy4tElslDAwIQX', function=Function(arguments='{"query":"개봉영화"}', name='Naver_News_Search'), type='function')])

In [36]:
name = tool_call_result.tool_calls[0].function.name
argument = tool_call_result.tool_calls[0].function.arguments

name,argument

('Naver_News_Search', '{"query":"개봉영화"}')

In [37]:
available_functions = {'Naver_News_Search': get_news, 'Tavily_Search': tavily_search}

name, arguments = tool_call_result.tool_calls[0].function.name, tool_call_result.tool_calls[0].function.arguments
search_result = available_functions[name](**json.loads(argument))

result = search_bot([
    {
        "role": "user",
        "content": """
요즘 새로 개봉한 영화는 무엇이 있나요? 네이버 검색으로 알려주세요
"""
    },
    tool_call_result, # tool_calls 메시지
    {
        "role": "tool",
        "content": search_result,
        # get_news(query = '영화')

        "tool_call_id":tool_call_result.tool_calls[0].id
    }
])

print(result.content)

요즘 새로 개봉한 영화 중에서 특히 흥행 중인 영화는 '좀비딸'입니다. 이 영화는 개봉 6일 만에 200만 관객을 돌파하며 올해 최고 흥행 속도를 기록하고 있습니다. 또한, '극장판 귀멸의 칼날: 무한성편'이 곧 개봉 예정이며, 중국에서는 일제 침략을 다룬 '731'이라는 영화가 개봉 예정입니다.

더 자세한 내용이나 다른 영화에 대해 알고 싶으시면 말씀해 주세요.


전체 과정을 하나의 함수로 만들 수 있습니다.   
LLM은 2번 실행해야 합니다.

In [39]:
def qa_bot(prompt):

    available_functions = {'Naver_News_Search': get_news, 'Tavily_Search': tavily_search}

    print('Prompt:',prompt)

    tool_call_result = search_bot([
        {
            "role": "user",
            "content": prompt
        }
    ])
    # 프롬프트를 받아 툴 사용 여부를 결정합니다.
    # 단순 대화라면 tool call을 생성하지 않습니다.

    print('---')
    print('News_Bot: Call ', end='')

    if tool_call_result.tool_calls: # tool_calls가 존재하면:

        name, arguments = tool_call_result.tool_calls[0].function.name, tool_call_result.tool_calls[0].function.arguments

        print(name,arguments)

        # 툴 실행
        search_result = available_functions[name](**json.loads(arguments))
        print('---')
        print('News_Bot:',end='')

        # 프롬프트 + 툴 요청 + 툴 실행 결과 전달
        response = search_bot([
        {
            "role": "user",
            "content": prompt
        },
        tool_call_result, # tool_calls 메시지
        {
            "role": "tool",
            "content": search_result,
            "tool_call_id":tool_call_result.tool_calls[0].id
        }],
        stream=True # 최종 결과는 스트리밍밍
        )

        for chunk in response:
            print(chunk.choices[0].delta.content, end='')
    else:
        print('Nothing')
        print(tool_call_result.content)

prompt = """Tavily에서 넷플릭스 신작 추천해줘."""

qa_bot(prompt)

Prompt: Tavily에서 넷플릭스 신작 추천해줘.
---
News_Bot: Call Tavily_Search {"query":"Netflix new releases recommendations","max_results":5}
---
News_Bot:최근 넷플릭스 신작 추천을 알려드릴게요.

1. KPop Demon Hunters
2. Mission: Impossible - Dead Reckoning
3. Oppenheimer
4. Barbarian
5. Wallace & Gromit: Vengeance Most Fowl
6. Nonnas

이 작품들은 넷플릭스 공식 사이트에서 새로 올라온 신작들입니다. 관심 있는 장르나 스타일에 맞춰 선택해 보시면 좋을 것 같아요. 더 자세한 정보가 필요하면 알려주세요!None

In [40]:
prompt = """네이버에서 왓챠 신작 추천해줘."""

qa_bot(prompt)

Prompt: 네이버에서 왓챠 신작 추천해줘.
---
News_Bot: Call Naver_News_Search {"query":"왓챠 신작"}
---
News_Bot:왓챠 신작으로 다음 작품들을 추천드립니다:

1. <임강선: 당신에게 스며든 진한 그리움> - 중국 드라마
2. <DOPE 마약 단속부 특수과> - 일본 드라마, 신종 마약 창궐을 막는 이야기
3. <마물>, <미션>, <사임당 빛의 일기>, <암향래: 복수의 향기> 등 다양한 장르의 영화와 드라마
4. <츠즈이 씨>, <동지: 너에게 닿은 계절>, <뜨거운 것이 좋아> - 일본 힐링 드라마, 중국 미스터리 로맨스, 고전 명작 영화
5. <더 헤드>, <미스터 로봇>, <낭만수급니: 사랑은 연재중> - 미스터리 스릴러, 애니메이션, 일본 범죄 수사 드라마
6. <그 겨울, 우리는>, <모모>, <허니 레몬 소다> - 중드, 대만 BL 드라마, 학원 애니메이션
7. <모던 타임즈>, <마운틴 닥터>, <하우스 오브 걸스>, <제7의 봉인> - 고전 명작 영화와 일본 드라마
8. <IGNITE>, <미소의 세상>, <거짓말의 온도> - 일본 다크 리걸 드라마 등
9. <인사팀 히토미>, <을의 연애>, <나만의 불청객>, <샹그릴라 프론티어 2nd Season> - 일본 드라마, BL 로맨스, 중국 청춘물, 게임 판타지 모험담

더 자세한 정보나 특정 장르를 원하시면 말씀해 주세요!None

In [41]:
qa_bot("회사 가기 싫어.")

Prompt: 회사 가기 싫어.
---
News_Bot: Call Nothing
회사 가기 싫을 때는 정말 힘들죠. 혹시 무슨 특별한 이유가 있나요? 스트레스가 많거나 피곤한 건 아닌지, 아니면 다른 고민이 있는지 이야기해주시면 도움이 될 수 있을 것 같아요.


# [심화] Parallel Tool Call
때로는, 하나의 메시지에서 여러 개의 Tool Call을 수행해야 하기도 합니다.   


In [42]:
# multiple tool call 필요

tool_call_result = search_bot([
    {
        "role": "user",
        "content": """
왓챠와 넷플릭스 신작 추천해줘"""
    }
])

rprint(tool_call_result)

ChatCompletionMessage(
    content=None,
    refusal=None,
    role='assistant',
    annotations=[],
    audio=None,
    function_call=None,
    tool_calls=[
        ChatCompletionMessageToolCall(
            id='call_IPJthPikpOHf1nJ8riHFiclX',
            function=Function(arguments='{"query": "왓챠 신작"}', name='Naver_News_Search'),
            type='function'
        ),
        ChatCompletionMessageToolCall(
            id='call_vSSO0oQFowCiuHJ8KqVIJbXk',
            function=Function(arguments='{"query": "넷플릭스 신작"}', name='Naver_News_Search'),
            type='function'
        )
    ]
)

이와 같은 경우, 반복문을 통해 여러 Tool Call의 결과를 전달합니다.  
각각의 id와 결과를 mapping합니다.

In [43]:
def qa_bot_v2(prompt, model = 'gpt-4.1-mini'):

    available_functions = {'Naver_News_Search': get_news, 'Tavily_Search': tavily_search}
    print('Prompt:',prompt)



    tool_call_result = search_bot([
        {
            "role": "user",
            "content": prompt
        },
    ])


    print('---')
    print('News_Bot: Call ', end='')

    if tool_call_result.tool_calls: # tool_calls가 존재하면:

        ### Tool Message 복수 입력

        tool_messages=[]

        # 여러 개의 tool_call에 대해, search_result 계산하여 리스트로 저장

        for tool_call in tool_call_result.tool_calls:
            name, arguments = tool_call.function.name, tool_call.function.arguments
            print(name,arguments)

            search_result = available_functions[name](**json.loads(arguments))

            print('---')
            print('Search_Bot:',end='')

            tool_messages.append(
                {"role": "tool","content": search_result,"tool_call_id":tool_call.id}
            )
        ####


        print("Call LLM")
        response = search_bot(
        [
        {
            "role": "user",
            "content": prompt
        },
        tool_call_result] + tool_messages,
        stream=True,
        model = model
        )
        for chunk in response:
            print(chunk.choices[0].delta.content, end='')
    else:
        print('Nothing')
        print(tool_call_result.content)


prompt = """넷플릭스 왓챠 신작 추천해줘."""

qa_bot_v2(prompt)

Prompt: 넷플릭스 왓챠 신작 추천해줘.
---
News_Bot: Call Naver_News_Search {"query": "넷플릭스 신작"}
---
Search_Bot:Naver_News_Search {"query": "왓챠 신작"}
---
Search_Bot:Call LLM
넷플릭스와 왓챠의 신작 중 추천할 만한 작품들을 알려드릴게요.

넷플릭스 신작 추천:
1. 에스피어: 변호사를 훑어보는 변호사들 (8월 3일 공개)
2. 트리거 (7월 25일 공개)
3. 오만과 편견 시리즈 (베넷 자매의 가혹사진 첫 공개)
4. 고백의 역사 (8월 공개 예정)
5. HOPE (2026년 여성 개봉 예정, 나홍진 감독 신작)

왓챠 신작 추천:
1. 폴스 (2025년 4월 공개 예정)
2. 하이픈 (7월 29일 공개)
3. 마인드 헌터 (7월 24일 공개)
4. 에스피어: 변호사를 훑어보는 변호사들 (넷플릭스와 동시 공개)
5. 다양한 한국 드라마 및 영화 신작 다수

이 중에서 관심 가는 작품을 골라 보시면 좋을 것 같아요. 더 자세한 정보나 특정 장르 추천 원하시면 말씀해 주세요!None

In [44]:
qa_bot_v2(prompt, model='gpt-4.1')

Prompt: 넷플릭스 왓챠 신작 추천해줘.
---
News_Bot: Call Naver_News_Search {"query": "넷플릭스 신작"}
---
Search_Bot:Naver_News_Search {"query": "왓챠 신작"}
---
Search_Bot:Call LLM
최신 넷플릭스와 왓챠 신작을 추천해드릴게요!

### 넷플릭스 신작 추천
1. **에스케이어: 변호사를 꿈꾸는 변호사들** (8월 3일 공개)  
   - 변호사를 꿈꾸는 이들의 성장과 도전을 그린 드라마.
2. **고백의 역사** (7월 29일 공개)  
   - 90년대 감성을 담은 로맨스 영화.
3. **트리거**  
   - 최근 공개된 액션 스릴러 드라마.
4. **오만과 편견**  
   - 가족사진을 소재로 한 따뜻한 드라마 시리즈.
5. **마인드헌터**  
   - 범죄 심리와 수사를 다룬 인기 시리즈의 신작.

자세한 정보: [넷플릭스 8월 신작 라인업](https://www.bntnews.co.kr/article/view/bnt202507310168)

---

### 왓챠 신작 추천
1. **하이퍼 프로젝트 하이쿠!!**  
   - 일본 인기 애니메이션 시리즈의 신작.
2. **명탐정 코난: 베이커가의 망령**  
   - 코난 극장판 시리즈 신작.
3. **슬로우 트레인**  
   - 일본 드라마 신작.
4. **사라진 첫사랑**  
   - 일본 BL 드라마로, 풋풋한 첫사랑을 그린 작품.
5. **대도시의 사랑법**  
   - 한국 퀴어 드라마 신작.

자세한 정보: [왓챠 신작 안내](http://www.movist.com/movist3d/view.asp?type=13&id=atc000000011770)

---

원하시는 장르나 더 궁금한 작품이 있으면 말씀해 주세요!None

한 번에 입력되는 툴 출력이 너무 길면, 할루시네이션이 발생하기도 하는데요.

이번에는 Parallel Tool Calling을 비활성화하고, 하나씩 실행해 보겠습니다.    
이 경우, Tool 호출 --> 메시지 전달 --> Tool 호출 --> 형태로 전달됩니다.

In [45]:
def search_bot_v2(messages, model = 'gpt-4.1-mini'):

    # 사용할 툴 펑션의 목록과 설명을 리스트로 전달
    # LLM이 스스로 description, name, parameter의 값을 통해 판단

    response = client.chat.completions.create(
        model = model,
        messages = messages,

        tools = tools,
        tool_choice = 'auto',

        temperature= 0,
        max_tokens= 1024,


        parallel_tool_calls = False,
        # 툴 동시 실행 대신 번갈아 실행하기 (Tool-->ToolMsg-->Tool-->ToolMsg-->...)
    )
    return response.choices[0].message

In [46]:
tool_call_result = search_bot_v2([
    {
        "role": "user",
        "content": """
넷플릭스 왓챠 신작 추천해줘.
"""
    }
])

rprint(tool_call_result)

ChatCompletionMessage(
    content=None,
    refusal=None,
    role='assistant',
    annotations=[],
    audio=None,
    function_call=None,
    tool_calls=[
        ChatCompletionMessageToolCall(
            id='call_9jmbLubKDeXffTZiUSbgNRbk',
            function=Function(arguments='{"query":"넷플릭스 신작"}', name='Naver_News_Search'),
            type='function'
        )
    ]
)

In [48]:
def qa_bot_v3(prompt, model = 'gpt-4.1-mini'):

    print('Prompt:',prompt)

    available_functions = {'Naver_News_Search': get_news, 'Tavily_Search': tavily_search}

    msgs = [
        {
            "role": "user",
            "content": prompt
        },
    ]

    tool_call_result = search_bot_v2(msgs)

    print('---')
    print('Search_Bot: Call ', end='')

    while tool_call_result.tool_calls:
    # tool_calls --> tool msg --> tool_calls 형태이므로 while로 처리

        msgs.append(tool_call_result)
        name, arguments = tool_call_result.tool_calls[0].function.name, tool_call_result.tool_calls[0].function.arguments
        print(name,arguments)

        search_result = available_functions[name](**json.loads(arguments))
        print('---')
        print('Search_Bot:',end='')

        msgs.append(
            {"role": "tool","content": search_result,"tool_call_id":tool_call_result.tool_calls[0].id}
        )
        tool_call_result = search_bot_v2(msgs, model = model)
        print("Call LLM")


    print(tool_call_result.content)


prompt = """넷플릭스, 왓챠 신작 추천해줘."""
qa_bot_v3(prompt)

Prompt: 넷플릭스, 왓챠 신작 추천해줘.
---
Search_Bot: Call Naver_News_Search {"query":"넷플릭스 신작"}
---
Search_Bot:Call LLM
Naver_News_Search {"query":"왓챠 신작"}
---
Search_Bot:Call LLM
넷플릭스 신작 추천:
1. '애마' - 이하늬 주연, 8월 18일 공개 예정
2. '에스콰이어: 변호사를 꿈꾸는 변호사들' - 8월 3일 공개
3. '고백의 역사' - 8월 29일 공개 예정
4. '트리거' - 총기 재난 액션 스릴러, 최근 공개
5. '나를 충전해줘' - 김영광 주연, 제작 확정

왓챠 신작 추천:
1. 'DOPE 마약 단속부 특수과' - 일본 드라마, 3분기 독점 공개
2. '임강선: 당신에게 스며든 진한 그리움' - 중국 드라마
3. '마물' - 일본 드라마
4. '더 헤드' 시즌1~2 - 미스터리 스릴러
5. '그 겨울, 우리는' - 중국 드라마
6. 'IGNITE' - 일본 다크 리걸 드라마
7. '인사팀 히토미' - 일본 드라마
8. '콩트가 시작된다' - 일본 드라마
9. '명탐정 코난: 베이커가의 망령' - 극장판 애니메이션

이 중에서 관심 있는 장르나 작품을 알려주시면 더 자세한 정보를 드릴 수 있습니다.


In [49]:
prompt = """Qwen LLama의 최신 버전 번호를 각각 검색하여 찾고, 한국어를 더 잘하는 모델이 무엇인지 비교해줘."""
qa_bot_v3(prompt)

Prompt: Qwen LLama의 최신 버전 번호를 각각 검색하여 찾고, 한국어를 더 잘하는 모델이 무엇인지 비교해줘.
---
Search_Bot: Call Tavily_Search {"query":"latest version number of Qwen Llama","max_results":5}
---
Search_Bot:Call LLM
Tavily_Search {"query":"latest version number of LLaMA","max_results":5}
---
Search_Bot:Call LLM
Tavily_Search {"query":"Qwen LLaMA Korean language performance comparison","max_results":5}
---
Search_Bot:Call LLM
Qwen LLaMA 최신 버전 정보:
- Qwen의 최신 버전은 Qwen3입니다. Qwen3-235B-A22B 모델이 대표적이며, 119개 언어와 방언을 지원합니다.
- LLaMA의 최신 버전은 Llama 4로, 2025년 4월에 출시되었습니다. Llama 4는 Scout와 Maverick이라는 모델을 포함하며, 멀티모달 지원과 긴 컨텍스트 길이를 특징으로 합니다.

한국어 성능 비교:
- Qwen은 다국어 지원에 강점을 가지고 있으며, 특히 아시아 언어에 강한 경향이 있습니다.
- LLaMA는 경량화와 빠른 추론에 중점을 두고 있으며, 영어 환경에서 더 우수한 성능을 보이는 경향이 있습니다.
- 한국어를 포함한 아시아 언어 처리에서는 Qwen이 더 나은 성능을 보일 가능성이 높습니다.

요약하면, 최신 버전은 Qwen3와 Llama 4이며, 한국어 성능 면에서는 Qwen이 더 우수할 가능성이 큽니다.


In [53]:
prompt = """한국의 국가대표 AI 팀으로 선정된 5개 팀을 찾고,
각 팀별 대표 LLM 모델 이름에 대해 검색해서 알려줘."""
qa_bot_v3(prompt)

Prompt: 한국의 국가대표 AI 팀으로 선정된 5개 팀을 찾고,
각 팀별 대표 LLM 모델 이름에 대해 검색해서 알려줘.
---
Search_Bot: Call Naver_News_Search {"query":"국가대표 AI 팀"}
---
Search_Bot:Call LLM
Tavily_Search {"query":"Naver AI model name","max_results":5}
---
Search_Bot:Call LLM
Tavily_Search {"query":"LG AI model name","max_results":5}
---
Search_Bot:Call LLM
Tavily_Search {"query":"SKT AI model name","max_results":5}
---
Search_Bot:Call LLM
Tavily_Search {"query":"NC AI model name","max_results":5}
---
Search_Bot:Call LLM
Tavily_Search {"query":"Upstage AI model name","max_results":5}
---
Search_Bot:Call LLM
한국의 국가대표 AI 팀으로 선정된 5개 팀과 각 팀별 대표 LLM 모델 이름은 다음과 같습니다.

1. 네이버 (Naver)
   - 대표 LLM 모델: HyperCLOVA X

2. LG AI 연구원 (LG AI Research)
   - 대표 LLM 모델: EXAONE (예: EXAONE Deep, EXAONE 4.0)

3. SK텔레콤 (SK Telecom, SKT)
   - 대표 LLM 모델: A.X LLM (예: AX 3.1 Lite, AX 4.0)

4. NC AI (NCSoft AI)
   - 대표 LLM 모델: VARCO (예: Varco Vision 2.0 1.7B)

5. 업스테이지 (Upstage)
   - 대표 LLM 모델: SOLAR (예: SOLAR-10.7B, SOLAR-0-70b)

이들 5개 팀은 정부의 '독자 